## Import

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
import json
from datetime import datetime
from tqdm import tqdm
import pytz
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [2]:
# Training and Testing directories
training_dir = os.path.join("Datasets", "Training")
testing_dir = os.path.join("Datasets", "Testing")
if not os.path.isdir(training_dir):
    raise Exception("ERROR: training dataset not found")
if not os.path.isdir(testing_dir):
    raise Exception("ERROR: testing dataset not found")

In [3]:
# iterate over all hashtag files 
for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        print(os.path.join(root, file))

Datasets/Training/tweets_#gohawks.txt
Datasets/Training/tweets_#gopatriots.txt
Datasets/Training/tweets_#nfl.txt
Datasets/Training/tweets_#patriots.txt
Datasets/Training/tweets_#sb49.txt
Datasets/Training/tweets_#superbowl.txt


In [4]:
# Initialize dictionaries and Unix times for Feb 1, 8 am and Feb 1, 8 pm. 
# Dictionary keys: hashtag.
# Dictionary values: [time of tweet (Unix), number of retweets for tweet, number of followers for tweeter]
# Each row in dictionary value is an individual tweet.

hashtag_dict_before = {}
hashtag_dict_during = {}
hashtag_dict_after = {}
start_unix_time = 1422806400 # 8 am, Feb 1, PST
end_unix_time = 1422849600 # 8 pm, Feb 1, PST
pst_tz = pytz.timezone('America/Los_Angeles')


In [5]:
""" Parse files to get necessary data """

for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        filename = os.path.splitext(file)[0].replace('tweets_#', '')
        print('Parsing {}...'.format(filename))
        
        hashtag_dict_before[filename] = []
        hashtag_dict_during[filename] = []
        hashtag_dict_after[filename] = []
        
        # open the file and read all lines:
        with open(os.path.join(root, file), "r", encoding="utf-8") as hashtag:
            # read line-by-line
            for line in hashtag:
                json_obj = json.loads(line)
                
                # get desired statistics
                citation_date = json_obj['citation_date'] # Unix time
                num_retweets = json_obj['metrics']['citations']['total'] # Number of retweets for this tweet
                num_followers = json_obj['author']['followers'] # Number of followers for tweeter
                
                # Check when tweet was made and add it to corresponding dictionary
                if citation_date < start_unix_time:
                    hashtag_dict_before[filename].append([citation_date, num_retweets, num_followers])
                elif citation_date > end_unix_time:
                    hashtag_dict_after[filename].append([citation_date, num_retweets, num_followers])
                else:
                    hashtag_dict_during[filename].append([citation_date, num_retweets, num_followers])
    print('done')

Parsing gohawks...
Parsing gopatriots...
Parsing nfl...
Parsing patriots...
Parsing sb49...
Parsing superbowl...
done


## Organize Data

##### Variables:
<span>
key = one of the hashtags <br \> <br \>
</span>

<span>
data_hashtag_before[key] = data before 2/1 8am, split into 1-hour windows (separated by hashtag) <br \>
data_hashtag_during[key] = data between 2/1 8am and 8pm, split into 5-min windows (separated by hashtag) <br \>
data_hashtag_after[key] = data after 2/1 8pm, split into 1-hour windows (separated by hashtag) <br \> <br \>
</span>

<span>
data_aggregate_before = data before 2/1 8am, split into 1-hour windows (all hashtags combined) <br \>
data_aggregate_during = data between 2/1 8am and 8pm, split into 5-min windows (all hashtags combined) <br \>
data_aggregate_after = data after 2/1 8pm, split into 1-hour windows (all hashtags combined) <br \> <br \>
</span>

<span>
data_hashtag_all[key] = all data, split into 1-hour windows (separated by hashtag) <br \> <br \>
</span>

<span>
data_all = all data, split into 1-hour windows (all hashtags combined) <br \>
</span>

In [6]:
# Explicitly list hashtags. 
# Convert each value in dictionary to numpy arrays.

hashtags = ['gohawks', 'gopatriots', 'nfl', 'patriots', 'sb49', 'superbowl']

for key in hashtags:
    hashtag_dict_before[key] = np.array(hashtag_dict_before[key])
    hashtag_dict_during[key] = np.array(hashtag_dict_during[key])
    hashtag_dict_after[key] = np.array(hashtag_dict_after[key])

In [7]:
# Find how many time windows there are

ftt = int(np.min([np.min(hashtag_dict_before[key][:,0]) for key in hashtags])) # first tweet time
ltt = int(np.max([np.max(hashtag_dict_after[key][:,0]) for key in hashtags])) # last tweet time

num_windows_before = int(np.max([((start_unix_time - ftt) // 3600) + 1 for key in hashtags]))
num_windows_during = int(np.max([((end_unix_time - start_unix_time) // 3600 * 12) for key in hashtags]))
num_windows_after = int(np.max([((ltt - end_unix_time) // 3600) + 1 for key in hashtags]))


In [8]:
""" Organize data into specific time periods:
     before 2/1 8am with 1-hour windows, 
     between 2/1 8am and 2/1 8pm with 5-min windows,
     and after 2/1 8pm with 1-hour windows """

# Initialize dictionary for each time frame.
data_hashtag_before = {}
data_hashtag_during = {}
data_hashtag_after = {}

# Iterate through each hashtag.
for key in hashtags:
    print(key)
    
    # Rename the dictionary value for readability
    temp_before = hashtag_dict_before[key]
    temp_during = hashtag_dict_during[key]
    temp_after = hashtag_dict_after[key]
    
    data_hashtag_before[key] = np.zeros((num_windows_before, 5)) # Initialize array: rows = time window, columns = feature
    num_followers_before = {} # Initialize dictionary to count # of followers for each tweet
    
    
    
    # Iterate through all elements before start time
    for i in range(np.shape(temp_before)[0]):
        # Get row number
        item_before = int(num_windows_before - 1 - ((start_unix_time - temp_before[i,0] - 1) // 3600))
        # Update first 3 elements (# of tweets, total # retweets, total # followers)
        data_hashtag_before[key][item_before] += np.array([1, int(temp_before[i, 1]), int(temp_before[i, 2]), 0, 0])
        # Get time of day (hour)
        dt_obj_pst = datetime.fromtimestamp(temp_before[i,0], pst_tz)
        data_hashtag_before[key][item_before][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        # Get number of followers
        if item_before not in num_followers_before.keys():
            num_followers_before[item_before] = []
        num_followers_before[item_before].append(temp_before[i,2])
    for i in num_followers_before.keys():
        data_hashtag_before[key][i][3] = np.max(num_followers_before[i])
        
        
    # Iterate through all elements during time
    data_hashtag_during[key] = np.zeros((num_windows_during, 5))
    num_followers_during = {}
    for i in range(np.shape(temp_during)[0]):
        item_during = int(((temp_during[i,0] - start_unix_time) * 12) // 3600)
        data_hashtag_during[key][item_during] += np.array([1, int(temp_during[i, 1]), int(temp_during[i, 2]), 0, 0])
        dt_obj_pst = datetime.fromtimestamp(temp_during[i,0], pst_tz)
        data_hashtag_during[key][item_during][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_during not in num_followers_during.keys():
            num_followers_during[item_during] = []
        num_followers_during[item_during].append(temp_during[i,2])
    for i in num_followers_during.keys():
        data_hashtag_during[key][i][3] = np.max(num_followers_during[i])
        
    # Iterate through all elements after end time
    data_hashtag_after[key] = np.zeros((num_windows_after, 5))
    num_followers_after = {}
    for i in range(np.shape(temp_after)[0]):
        item_after = int((temp_after[i,0] - end_unix_time) // 3600)
        data_hashtag_after[key][item_after] += np.array([1, int(temp_after[i, 1]), int(temp_after[i, 2]), 0, 0])
        dt_obj_pst = datetime.fromtimestamp(temp_after[i,0], pst_tz)
        data_hashtag_after[key][item_after][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_after not in num_followers_after.keys():
            num_followers_after[item_after] = []
        num_followers_after[item_after].append(temp_after[i,2])
    for i in num_followers_after.keys():
        data_hashtag_after[key][i][3] = np.max(num_followers_after[i])
        
print('done')

gohawks
gopatriots
nfl
patriots
sb49
superbowl
done


In [13]:
""" Aggregate data within each time period by combining all hashtags. """

# Initialize aggregated data variables
data_aggregate_before = np.zeros([num_windows_before, 5])
data_aggregate_during = np.zeros([num_windows_during, 5])
data_aggregate_after = np.zeros([num_windows_after, 5])

# Sum the # of tweets, total # of retweets, and # of followers
for key in hashtags:
    data_aggregate_before[:,0:3] += data_hashtag_before[key][:,0:3]
    data_aggregate_during[:,0:3] += data_hashtag_during[key][:,0:3]
    data_aggregate_after[:,0:3] += data_hashtag_after[key][:,0:3]
# Find the max # of followers for each
data_aggregate_before[:,3] = np.amax([data_hashtag_before[key][:,3] for key in hashtags], axis=0)
data_aggregate_during[:,3] = np.amax([data_hashtag_during[key][:,3] for key in hashtags], axis=0)
data_aggregate_after[:,3] = np.amax([data_hashtag_after[key][:,3] for key in hashtags], axis=0)

# Copy over the same time frames
data_aggregate_before[:,4] = data_hashtag_before['superbowl'][:,4]
data_aggregate_during[:,4] = data_hashtag_during['superbowl'][:,4]
data_aggregate_after[:,4] = data_hashtag_after['superbowl'][:,4]

In [14]:
""" Get data for the whole time frame with 1-hour windows, separated by hashtag """

# Initialize dictionary to store data.
# Key: hashtag
# Value: data separated by 1-hour time windows
data_hashtag_all = {}

for key in hashtags: # Iterate through all hashtags
    temp_during = np.zeros([12, 5]) # Initialize array to store data in the middle time period
    # Combine data in the middle time period
    for i in range(np.shape(data_hashtag_during[key])[0]):
        hour = int(data_hashtag_during[key][i,4] - 8)
        temp_during[hour, :3] += data_hashtag_during[key][i, :3]
        if not i % 12:
            temp_during[hour, 3] = np.max(data_hashtag_during[key][i:(i+12), 3])
            temp_during[hour, 4] = data_hashtag_during[key][i,4]
    data_hashtag_all[key] = np.vstack((data_hashtag_before[key], temp_during, data_hashtag_after[key]))

In [15]:
""" Combine data for the whole time frame from all hashtags """

data_all = np.zeros([587, 5])
for key in hashtags:
    data_all += data_hashtag_all[key]

### Neural Network

In [16]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [17]:
# Define layer sizes.
layer_sizes = [(50, 50), (100, 100), (100, 100, 100), (100, 100, 100, 100), 10*(50,), 10*(100,)]

In [18]:
# Function to get MSE for a given neural network model.

def analyze_nn(nn, X, y):
    mses_per_fold = []
    mses_per_fold_train = []
    kf = KFold(10)
    for trainset, testset in kf.split(X):
        X_train, y_train = X[trainset], y[trainset]
        X_test, y_test = X[testset], y[testset]
        nn.fit(X_train, y_train)
        predicted = nn.predict(X_test)
        mses_per_fold.append(mean_squared_error(y_test, predicted))
        mses_per_fold_train.append(mean_squared_error(y_train, nn.predict(X_train)))
    #     print(mean_squared_error(y_test, predicted))
    avg_mse = np.mean(mses_per_fold)
    avg_mse_train = np.mean(mses_per_fold_train)
    print('Layer size {} MSE:\n   val = {} \n train = {}'.format(size, np.around(avg_mse, 2), np.around(avg_mse_train, 2)))
    
    return avg_mse, avg_mse_train

#### No preprocessing

In [15]:
# Analyze tweets before start time.

# Define train data and targets
y = data_all[1:,0] # Number of tweets (except first)
X = np.delete(data_all, -1, 0) # Delete last row

# X = X_before
# y = y_before

print('X shape:', X.shape)
print('y shape:', y.shape)

mses = []
mses_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses.append(avg_mse)
    mses_train.append(avg_mse_train)

X shape: (586, 5)
y shape: (586,)
Layer size (50, 50) MSE:
   val = 133015103014.18 
 train = 2207226555.42
Layer size (100, 100) MSE:
   val = 15844716815.04 
 train = 30054698324.78
Layer size (100, 100, 100) MSE:
   val = 14112975642.27 
 train = 2126070891775.56
Layer size (100, 100, 100, 100) MSE:
   val = 2821068693.02 
 train = 23485684975.48
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 684214921.64 
 train = 2484996332.98
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 650577997.55 
 train = 2851214279.48


In [16]:
print(np.sqrt(mses))

[364712.35654167 125875.79916346 118798.04561636  53113.73356318
  26157.50220572  25506.43051367]


#### Standard scaler preprocessing

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
scaler = StandardScaler()

In [19]:
y = data_all[1:,0] # Number of tweets (except first)
X = np.delete(data_all, -1, 0) # Delete last row

X = scaler.fit_transform(X) # Transform data
y = y

print('X shape:', X.shape)
print('y shape:', y.shape)

mses = []
mses_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses.append(avg_mse)
    mses_train.append(avg_mse_train)

X shape: (586, 5)
y shape: (586,)
Layer size (50, 50) MSE:
   val = 779705827.18 
 train = 643736487.08
Layer size (100, 100) MSE:
   val = 673236649.94 
 train = 373934473.11
Layer size (100, 100, 100) MSE:
   val = 258232605.09 
 train = 229071346.09
Layer size (100, 100, 100, 100) MSE:
   val = 359969571.59 
 train = 191698219.79
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 687035152.05 
 train = 115044266.39
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 602909849.65 
 train = 87523650.19


In [20]:
print(np.sqrt(mses))

[27923.21305265 25946.80423359 16069.61745298 18972.86408511
 26211.35540269 24554.22264404]


In [23]:
y = data_all[1:,0] # Number of tweets (except first)
X = np.delete(data_all, -1, 0) # Delete last row

X = scaler.fit_transform(X) # Transform data
y = y

layer_sizes = [(50,50,50), (100,100,100), (150,150,150), (200,200,200)]

print('X shape:', X.shape)
print('y shape:', y.shape)

mses = []
mses_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses.append(avg_mse)
    mses_train.append(avg_mse_train)

X shape: (586, 5)
y shape: (586,)
Layer size (50, 50, 50) MSE:
   val = 493703899.18 
 train = 244183617.51
Layer size (100, 100, 100) MSE:
   val = 261863410.38 
 train = 229693258.82
Layer size (150, 150, 150) MSE:
   val = 270112436.37 
 train = 220036898.54
Layer size (200, 200, 200) MSE:
   val = 285625289.79 
 train = 207547415.99


In [24]:
print(np.sqrt(mses))

[22219.44866948 16182.19423887 16435.09769898 16900.4523546 ]


#### Grid search

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
y_before = data_aggregate_before[1:,0] # Number of tweets (except first)
X_before = np.delete(data_aggregate_before, -1, 0) # Delete last row

X = scaler.fit_transform(X_before) # Transform data
y = y_before

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,),
                           2*(50,), 2*(100,), 2*(150,), 2*(200,), 2*(250,), 2*(300,), 
                           3*(50,), 3*(100,), 3*(150,), 3*(200,), 3*(250,), 3*(300,),
                           4*(50,), 4*(100,), 4*(150,), 4*(200,), 4*(250,), 4*(300,)]
}

nn = MLPRegressor(activation='relu', solver='adam', alpha=0.001)
clf_before = GridSearchCV(nn, param_grid, cv=10, scoring='neg_mean_squared_error')
clf_before.fit(X, y)

/Users/dmwang626/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,), (50, 50), (100, 100), (150, 150), (200, 200), (250, 250), (300, 300), (50, 50, 50), (100, 100, 100), (150, 150, 150), (200, 200, 200), (250, 250, 250), (300, 300, 300), (50, 50, 50, 50), (100, 100, 100, 100), (150, 150, 150, 150), (200, 200, 200, 200), (250, 250, 250, 250), (300, 300, 300, 300)]},
       pre_dispatch='2*n_jobs', refit=T

In [28]:
print(clf.best_params_)
print(clf.best_score_)

{'hidden_layer_sizes': (100, 100, 100)}
-261706917.5742354


In [29]:
y_during = data_aggregate_during[1:,0] # Number of tweets (except first)
X_during = np.delete(data_aggregate_during, -1, 0) # Delete last row

X = scaler.fit_transform(X_during) # Transform data
y = y_during

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,),
                           2*(50,), 2*(100,), 2*(150,), 2*(200,), 2*(250,), 2*(300,), 
                           3*(50,), 3*(100,), 3*(150,), 3*(200,), 3*(250,), 3*(300,),
                           4*(50,), 4*(100,), 4*(150,), 4*(200,), 4*(250,), 4*(300,)]
}

nn = MLPRegressor(activation='relu', solver='adam', alpha=0.001)
clf_during = GridSearchCV(nn, param_grid, cv=10, scoring='neg_mean_squared_error')
clf_during.fit(X, y)

/Users/dmwang626/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,), (50, 50), (100, 100), (150, 150), (200, 200), (250, 250), (300, 300), (50, 50, 50), (100, 100, 100), (150, 150, 150), (200, 200, 200), (250, 250, 250), (300, 300, 300), (50, 50, 50, 50), (100, 100, 100, 100), (150, 150, 150, 150), (200, 200, 200, 200), (250, 250, 250, 250), (300, 300, 300, 300)]},
       pre_dispatch='2*n_jobs', refit=T

In [30]:
print(clf_during.best_params_)
print(clf_during.best_score_)

{'hidden_layer_sizes': (300, 300, 300, 300)}
-25750129.554445647


In [31]:
y_after = data_aggregate_after[1:,0] # Number of tweets (except first)
X_after = np.delete(data_aggregate_after, -1, 0) # Delete last row

X = scaler.fit_transform(X_after) # Transform data
y = y_after

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,),
                           2*(50,), 2*(100,), 2*(150,), 2*(200,), 2*(250,), 2*(300,), 
                           3*(50,), 3*(100,), 3*(150,), 3*(200,), 3*(250,), 3*(300,),
                           4*(50,), 4*(100,), 4*(150,), 4*(200,), 4*(250,), 4*(300,)]
}

nn = MLPRegressor(activation='relu', solver='adam', alpha=0.001)
clf_after = GridSearchCV(nn, param_grid, cv=10, scoring='neg_mean_squared_error')
clf_after.fit(X, y)

/Users/dmwang626/anaconda/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,), (50, 50), (100, 100), (150, 150), (200, 200), (250, 250), (300, 300), (50, 50, 50), (100, 100, 100), (150, 150, 150), (200, 200, 200), (250, 250, 250), (300, 300, 300), (50, 50, 50, 50), (100, 100, 100, 100), (150, 150, 150, 150), (200, 200, 200, 200), (250, 250, 250, 250), (300, 300, 300, 300)]},
       pre_dispatch='2*n_jobs', refit=T

In [32]:
print(clf_after.best_params_)
print(clf_after.best_score_)

{'hidden_layer_sizes': (300, 300, 300, 300)}
-1104433.9605698008
